**Google stock prices dataset and design a time series analysis and prediction system using RNN.**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv(r'/content/drive/MyDrive/Colab_Notebooks/4/Google_Stock_Price_Train.csv')

In [ ]:
train_df

,Date,Open,High,Low,Close,Volume
0,1/3/2012,325.25,332.83,324.97,663.59,"7,380,500"
1,1/4/2012,331.27,333.87,329.08,666.45,"5,749,400"
2,1/5/2012,329.83,330.75,326.89,657.21,"6,590,300"
3,1/6/2012,328.34,328.77,323.68,648.24,"5,405,900"
4,1/9/2012,322.04,322.29,309.46,620.76,"11,688,800"
...,...,...,...,...,...,...
1253,12/23/2016,790.90,792.74,787.28,789.91,"623,400"
1254,12/27/2016,790.68,797.86,787.66,791.55,"789,100"
1255,12/28/2016,793.70,794.23,783.20,785.05,"1,153,800"
1256,12/29/2016,783.33,785.93,778.92,782.79,"744,300"


In [ ]:
test_df = pd.read_csv(r'/content/drive/MyDrive/Colab_Notebooks/4/Google_Stock_Price_Test.csv')

With the feature of google colab to plot graphs, below graphs are drawn.

In [ ]:
test_df

,Date,Open,High,Low,Close,Volume
0,1/3/2017,778.81,789.63,775.80,786.14,"1,657,300"
1,1/4/2017,788.36,791.34,783.16,786.90,"1,073,000"
2,1/5/2017,786.08,794.48,785.02,794.02,"1,335,200"
3,1/6/2017,795.26,807.90,792.20,806.15,"1,640,200"
4,1/9/2017,806.40,809.97,802.83,806.65,"1,272,400"
5,1/10/2017,807.86,809.13,803.51,804.79,"1,176,800"
6,1/11/2017,805.00,808.15,801.37,807.91,"1,065,900"
7,1/12/2017,807.14,807.39,799.17,806.36,"1,353,100"
8,1/13/2017,807.48,811.22,806.69,807.88,"1,099,200"
9,1/17/2017,807.08,807.14,800.37,804.61,"1,362,100"


In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    20 non-null     object 
 1   Open    20 non-null     float64
 2   High    20 non-null     float64
 3   Low     20 non-null     float64
 4   Close   20 non-null     float64
 5   Volume  20 non-null     object 
dtypes: float64(4), object(2)
memory usage: 1.1+ KB


In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
train_df['Close'] = train_df['Close'].astype(str).str.replace(',', '').astype(float)
test_df['Close'] = test_df['Close'].astype(str).str.replace(',', '').astype(float)

In [ ]:
train_scaler = MinMaxScaler()
train_df['Normalized Close'] = train_scaler.fit_transform(train_df['Close'].values.reshape(-1, 1))
test_scaler = MinMaxScaler()
test_df['Normalized Close'] = test_scaler.fit_transform(test_df['Close'].values.reshape(-1, 1))

In [ ]:
train_df['Normalized Close']

0       0.237573
1       0.241514
2       0.228781
3       0.216419
4       0.178548
          ...   
1253    0.411656
1254    0.413916
1255    0.404958
1256    0.401844
1257    0.386726
Name: Normalized Close, Length: 1258, dtype: float64

In [ ]:
train_df['Normalized Close'].values[:-1]

array([0.23757287, 0.24151427, 0.22878051, ..., 0.41391618, 0.40495845,
       0.40184391])

In [ ]:
x_train = train_df['Normalized Close'].values[:-1].reshape(-1, 1, 1)
y_train = train_df['Normalized Close'].values[1:].reshape(-1, 1, 1)
x_test = test_df['Normalized Close'].values[:-1].reshape(-1, 1, 1)
y_test = test_df['Normalized Close'].values[1:].reshape(-1, 1, 1)

In [ ]:
print("x_train shape: ",x_train.shape)
print("y_train shape: ",y_train.shape)
print("x_test shape: ",x_test.shape)
print("y_test shape: ",y_test.shape)

x_train shape:  (1257, 1, 1)
y_train shape:  (1257, 1, 1)
x_test shape:  (19, 1, 1)
y_test shape:  (19, 1, 1)


In [ ]:
x_train

array([[[0.23757287]],

       [[0.24151427]],

       [[0.22878051]],

       ...,

       [[0.41391618]],

       [[0.40495845]],

       [[0.40184391]]])

In [ ]:
test_df

,Date,Open,High,Low,Close,Volume,Normalized Close
0,1/3/2017,778.81,789.63,775.80,786.14,"1,657,300",0.000000
1,1/4/2017,788.36,791.34,783.16,786.90,"1,073,000",0.015344
2,1/5/2017,786.08,794.48,785.02,794.02,"1,335,200",0.159095
3,1/6/2017,795.26,807.90,792.20,806.15,"1,640,200",0.403998
4,1/9/2017,806.40,809.97,802.83,806.65,"1,272,400",0.414092
5,1/10/2017,807.86,809.13,803.51,804.79,"1,176,800",0.376539
6,1/11/2017,805.00,808.15,801.37,807.91,"1,065,900",0.439532
7,1/12/2017,807.14,807.39,799.17,806.36,"1,353,100",0.408237
8,1/13/2017,807.48,811.22,806.69,807.88,"1,099,200",0.438926
9,1/17/2017,807.08,807.14,800.37,804.61,"1,362,100",0.372905


In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date              20 non-null     object 
 1   Open              20 non-null     float64
 2   High              20 non-null     float64
 3   Low               20 non-null     float64
 4   Close             20 non-null     float64
 5   Volume            20 non-null     object 
 6   Normalized Close  20 non-null     float64
dtypes: float64(5), object(2)
memory usage: 1.2+ KB


Model Building

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [ ]:
model = Sequential()
model.add(LSTM(4, input_shape=(1, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 4)                 96        
                                                                 
 dense (Dense)               (None, 1)                 5         
                                                                 
Total params: 101 (404.00 Byte)
Trainable params: 101 (404.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(x_train, y_train, epochs=100, batch_size=1, verbose=1)

Epoch 1/100
1257/1257 [==============================] - 15s 8ms/step - loss: 0.0269
Epoch 2/100
1257/1257 [==============================] - 5s 4ms/step - loss: 0.0018
Epoch 3/100
1257/1257 [==============================] - 5s 4ms/step - loss: 7.7435e-04
Epoch 4/100
1257/1257 [==============================] - 5s 4ms/step - loss: 7.6381e-04
Epoch 5/100
1257/1257 [==============================] - 5s 4ms/step - loss: 7.7490e-04
Epoch 6/100
1257/1257 [==============================] - 5s 4ms/step - loss: 7.5474e-04
Epoch 7/100
1257/1257 [==============================] - 5s 4ms/step - loss: 7.6777e-04
Epoch 8/100
1257/1257 [==============================] - 6s 5ms/step - loss: 7.6263e-04
Epoch 9/100
1257/1257 [==============================] - 5s 4ms/step - loss: 7.5643e-04
Epoch 10/100
1257/1257 [==============================] - 5s 4ms/step - loss: 7.6433e-04
Epoch 11/100
1257/1257 [==============================] - 6s 5ms/step - loss: 7.7534e-04
Epoch 12/100
1257/1257 [=============

Evaluating Model

In [ ]:
test_loss = model.evaluate(x_test, y_test)
print('Testing loss: ', test_loss)

1/1 [==============================] - 0s 431ms/step - loss: 0.0253
Testing loss:  0.02531779371201992


Testing Model

In [ ]:
y_pred = model.predict(x_test)

1/1 [==============================] - 0s 396ms/step


In [ ]:
y_test_actual = test_scaler.inverse_transform(y_test.reshape(-1, 1))
y_pred_actual = test_scaler.inverse_transform(y_pred.reshape(-1, 1))

In [ ]:
i=3

In [ ]:
print("Actual value: {:.2f}".format(y_test_actual[i][0]))
print("Predicted value: {:.2f}".format(y_pred_actual[i][0]))

Actual value: 806.65
Predicted value: 806.18


In [ ]:
x_test[1][0]

array([0.01534424])

In [ ]:
value = [[[0.41409247]]]
pred = model.predict(value)

value[0][0][0]

1/1 [==============================] - 0s 38ms/step


0.41409247

In [ ]:
y_pred_actual = test_scaler.inverse_transform(pred.reshape(-1, 1))

In [ ]:
a = [0,1,2,3,4,5,6,7,8,9,10]
print(a[1:])

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [ ]:
y_pred_actual

array([[806.68756]], dtype=float32)